In [11]:
from ttsxai.utils import jupyter_utils
# jupyter_utils.register_always_reload(packages=[
#     'ttsxai.interfaces',
#     # 'ttsxai.models.text2mel',
#     # 'ttsxai.models.mel2wave',
#     # 'ttsxai.utils',
# ])
jupyter_utils.register_always_reload(modules=[
    'ttsxai.manifold',
    # 'ttsxai.models.text2mel.tacotron2',
    # 'ttsxai.models.text2mel.fastspeech2',
    # 'ttsxai.utils.plot_utils',
    # 'ttsxai.pitch_calculator',
    # 'ttsxai.energy_calculator'
])

In [2]:
import os
import numpy as np
from matplotlib import pyplot as plt

from tacotron2.text import _id_to_symbol, _symbol_to_id

data_analysis_dir = "/nas/users/dahye/kw/tts/ttsxai/data_analysis/analyze_prosody"
data = np.load(os.path.join(data_analysis_dir, 'phone_data.npz'), allow_pickle=True)

In [3]:
phone_data = {}
for token, phone_data_item in data.items():
    phone_data[token] = phone_data_item.item()

In [35]:
from ttsxai.manifold import Manifold

In [39]:
manifold_dict = {}
for token in phone_data.keys():
    # token = int(token)
    print(token, _id_to_symbol[int(token)], len(phone_data[token]['duration']))
    manifold = Manifold(k=3, score_type='realism')
    feats_real = np.array(phone_data[token]['activations'][:2000])
    manifold.compute_manifold(feats_real)
    manifold_dict[token] = manifold

118 @M 23807
97 @ER0 16788
113 @IY1 13916
119 @N 68413
73 @AH0 78316
11   209112
78 @AO1 13532
146 @Z 28036
144 @W 20557
77 @AO0 1493
117 @L 32919
90 @D 43546
105 @G 5978
102 @EY1 16611
143 @V 19607
70 @AE1 19781
106 @HH 13811
98 @ER1 6766
129 @P 20263
109 @IH1 25368
145 @Y 4423
91 @DH 29317
133 @T 65830
116 @K 27897
141 @UW1 13672
79 @AO2 978
66 @AA1 15576
108 @IH0 25787
130 @R 40369
87 @AY2 1445
120 @NG 7240
7 . 10008
115 @JH 4842
74 @AH1 19635
131 @S 43528
123 @OW1 8144
94 @EH1 23225
112 @IY0 13127
89 @CH 4717
132 @SH 7982
67 @AA2 994
82 @AW1 3821
6 , 15150
104 @F 16967
95 @EH2 2253
88 @B 15611
86 @AY1 10499
114 @IY2 854
137 @UH1 2655
110 @IH2 2978
122 @OW0 1589
75 @AH2 429
134 @TH 3565
127 @OY1 796
85 @AY0 380
101 @EY0 707
136 @UH0 36
147 @ZH 612
65 @AA0 324
103 @EY2 1570
124 @OW2 595
142 @UW2 608
83 @AW2 406
9 ; 727
71 @AE2 1207
140 @UW0 875
3 ' 236
93 @EH0 900
4 ( 167
5 ) 167
8 : 177
69 @AE0 509
1 - 277
138 @UH2 120
10 ? 76
99 @ER2 169
128 @OY2 49
81 @AW0 39
126 @OY0 8
2 ! 46


In [40]:
import os
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

import torch
import torch.nn as nn
import torch.nn.functional as F

import neurox.interpretation.utils as utils
import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe

from ttsxai.utils.utils import read_ljs_metadata


log_dir = '/nas/users/dahye/kw/tts/ttsxai/logs/probe_tacotron2_duration'
data_activation_dir = "/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow"


In [42]:
import IPython.display as ipd
from matplotlib import pyplot as plt

import torch

from ttsxai.utils.utils import read_ljs_metadata
from ttsxai.utils import plot_utils
from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = '/nas/users/dahye/kw/tts/ttsxai/data/LJSpeech/metadata.csv'
# text_dict = read_ljs_metafile(data_dir)
text_dict = read_ljs_metadata(mode='train')

text2mel = get_text2mel('tacotron2', device)
mel2wave = get_mel2wave('waveglow', device)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/nas/users/dahye/kw/tts/ttsxai/src/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


In [43]:
tts = TTSInterface(
    device=device,
    text2mel=text2mel,
    mel2wave=mel2wave 
).to(device)

text = text_dict['LJ001-0002']
output_dict = tts.forward(text)
wave = output_dict['wave']

ipd.Audio(wave, rate=tts.sampling_rate)

(80, 168)


/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


In [44]:
X_test = np.concatenate(list(output_dict['activations'].values()), axis=1)

In [45]:
# load pre-trained probe
probe = linear_probe.LinearProbe(X_test.shape[1], 1)
probe.load_state_dict(
    torch.load(os.path.join(log_dir, 'models', 'probe.pth')))

<All keys matched successfully>

In [53]:
output_dict['token'][26]

7

In [54]:
len(phone_data['7']['duration'])

10008

In [52]:
X_test.shape

(27, 2048)

In [58]:
manifold_dict['7']

In [65]:
idx = 26
factor = 5
scale = 0.1
x = torch.tensor(X_test[idx][None], dtype=torch.float32)
# layer_k_X_test = ablation.filter_activations_by_layers(X_test, [3], 4)
# x = torch.tensor(layer_k_X_test[idx][None], dtype=torch.float32)
# x = torch.tensor(X, dtype=torch.float32)
init_v = probe(x).squeeze()

manifold = manifold_dict[str(output_dict['token'][idx])]

for _ in range(100):
    print(print(torch.exp(probe(x)), manifold.score(x.numpy())))
    with torch.enable_grad():
        x.requires_grad_()

        duration = probe(x).squeeze()
           
        # print(duration)
        grad = torch.autograd.grad(
            [duration], 
            [x]
        )[0]

        x = x.detach()
    if factor > 1:
        x = x + scale * grad
        if duration > init_v + np.log(factor):
            break
    else:
        x = x - scale * grad
        if duration < init_v + np.log(factor):
            break

    # if duration 
    # print(torch.exp(probe(x)), manifold.score(x.numpy()))
print(x)

tensor([[21.9276]], grad_fn=<ExpBackward0>) [3142428.8278335]
None
tensor([[24.1691]], grad_fn=<ExpBackward0>) [47.03975475]
None
tensor([[26.6399]], grad_fn=<ExpBackward0>) [23.51999529]
None
tensor([[29.3632]], grad_fn=<ExpBackward0>) [15.6800231]
None
tensor([[32.3649]], grad_fn=<ExpBackward0>) [11.76002708]
None
tensor([[35.6734]], grad_fn=<ExpBackward0>) [9.40802639]
None
tensor([[39.3202]], grad_fn=<ExpBackward0>) [7.8400246]
None
tensor([[43.3398]], grad_fn=<ExpBackward0>) [6.72002269]
None
tensor([[47.7702]], grad_fn=<ExpBackward0>) [5.88002091]
None
tensor([[52.6536]], grad_fn=<ExpBackward0>) [5.22668597]
None
tensor([[58.0362]], grad_fn=<ExpBackward0>) [4.70401797]
None
tensor([[63.9691]], grad_fn=<ExpBackward0>) [4.27638042]
None
tensor([[70.5084]], grad_fn=<ExpBackward0>) [3.92001569]
None
tensor([[77.7163]], grad_fn=<ExpBackward0>) [3.61847626]
None
tensor([[85.6609]], grad_fn=<ExpBackward0>) [3.36001386]
None
tensor([[94.4178]], grad_fn=<ExpBackward0>) [3.13601309]
None
t

In [62]:
x.numpy()

array([[-0.00082307,  0.0031323 , -0.00103865, ...,  0.08529945,
         0.5608754 , -0.0431802 ]], dtype=float32)

In [63]:
manifold.score(x.numpy())

array([2.61334453])

# train flow

In [66]:
# some imports and plot definitions

import torch
import numpy as np
import scipy
import os
from tqdm import tqdm
from scipy.optimize import NonlinearConstraint

import torch.optim as optim
import torch.nn as nn

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc

rc('text', usetex=True)
plt.rc('font', family='serif')

plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": ["Nimbus Roman"],
    "mathtext.fontset": "cm",
    "font.size": 30,
})

label_font_size = 32
title_font_size = 32

# make directories
directories = ["models", "plots", "results"]
for d in directories:
    if not os.path.exists(d):
        os.makedirs(d)

In [75]:
# GPU 사용 가능 여부 확인 및 device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [76]:
# definition of latent distribution
latent_distr = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(2048, device=device), torch.eye(2048, device=device))

In [77]:
# definition of the normalizing flow

nets = lambda: nn.Sequential(nn.Linear(2048, 256), nn.LeakyReLU(), 
                             nn.Linear(256, 256), nn.LeakyReLU(), 
                             nn.Linear(256, 2048), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(2048, 256), nn.LeakyReLU(), 
                             nn.Linear(256, 256), nn.LeakyReLU(), 
                             nn.Linear(256, 2048))


class RealNVP(torch.nn.Module):
    def __init__(self):
        super(RealNVP, self).__init__()
        # mask = torch.from_numpy(np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0], 
        #                                   [0, 0, 1], [1, 0, 0], [0, 1, 0]] * 2).astype(np.float32))
        # self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(mask))])
        self.s = torch.nn.ModuleList([nets() for _ in range(len(mask))])

    def forward(self, z):
        x = z
        for i in range(len(self.t)):
            # x_ = x * self.mask[i]
            # s = self.s[i](x_) * (1 - self.mask[i])
            # t = self.t[i](x_) * (1 - self.mask[i])
            # x = x_ + (1 - self.mask[i]) * (x * torch.exp(s) + t)
            x_ = x
            s = self.s[i](x_)
            t = self.t[i](x_)
            x = x_ + (x * torch.exp(s) + t)
        return x

    def reverse(self, x):
        log_det_J, z = x.new_zeros(x.shape[0]), x
        for i in reversed(range(len(self.t))):
            # z_ = self.mask[i] * z
            # s = self.s[i](z_) * (1 - self.mask[i])
            # t = self.t[i](z_) * (1 - self.mask[i])
            # z = (1 - self.mask[i]) * (z - t) * torch.exp(-s) + z_
            z_ = z
            s = self.s[i](z_) 
            t = self.t[i](z_) 
            z = (z - t) * torch.exp(-s) + z_
            log_det_J -= s.sum(dim=1)
            
        return z, log_det_J

In [ ]:
# funtion to train the flow

def train_flow(flow, latent_distr, target_distr, batch_size, epochs, optimizer, save_as, best_loss = np.inf):
    avg_loss = 0
    flow.train()
    with tqdm(total=epochs) as progress_bar:
        for epoch in range(epochs):
            # samples = target_distr.sample(batch_size)
            samples = target_distr.sample(batch_size).to(device)
            z, log_det = flow.reverse(samples)
            log_prob = latent_distr.log_prob(z)
            loss = -(log_det.mean() + log_prob.mean())
            flow.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss = moving_avg(avg_loss, loss, epoch + 1).item()
            progress_bar.set_postfix(loss=avg_loss)
            progress_bar.update(1)

            if (epoch+1)%100==0:
                flow.eval()
                avg_loss_val = 0
                for epoch_val in range(100):
                    # samples = target_distr.sample(batch_size)
                    samples = target_distr.sample(batch_size).to(device)
                    z, log_det = flow.reverse(samples)
                    log_prob = latent_distr.log_prob(z)
                    loss = -(log_det.mean() + log_prob.mean())
                    avg_loss_val = moving_avg(avg_loss_val, loss, epoch_val + 1).item()

                if avg_loss_val<best_loss:
                    best_loss=avg_loss_val
                    torch.save(flow, save_as)
                flow.train()
                    
    flow.eval()
    avg_loss = 0
    with tqdm(total=100) as progress_bar:
        for epoch in range(100):
            # samples = target_distr.sample(batch_size)
            samples = target_distr.sample(batch_size).to(device)
            z, log_det = flow.reverse(samples)
            log_prob = latent_distr.log_prob(z)
            loss = -(log_det.mean() + log_prob.mean())
            avg_loss = moving_avg(avg_loss, loss, epoch + 1).item()
            progress_bar.set_postfix(loss=avg_loss)
            progress_bar.update(1)

    return 1

def moving_avg(current_avg, new_value, idx):
    return current_avg + (new_value - current_avg) / idx